In [1]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
!pip install facebook-business

import psycopg2

from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.ad import Ad
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 10.1MB 19.5MB/s 
  Created wheel for curlify: filename=curlify-2.2.1-cp36-none-any.whl size=2660 sha256=aedfe6ea925aae1b6997d939085e01884f649e3c5ceb22a60e0d5ebc3922a7ad
  Stored in directory: /root/.cache/pip/wheels/b7/5e/e5/6f90a2b21faf7b71748fcba58920ad77850c25b8f70927d418
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=324b81e48ffe00166a4c456b8bfefea5959f1d01079d38c09ce1f68d9ca540f6
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built curlify pycountry


/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def store_object(object_data, table):
    fields = ','.join(object_data.keys())
    values = ','.join(map(lambda x:f"%({x})s",object_data.keys()))
    query =   f"INSERT INTO {table} ({fields}) values ({values})"
    cur.execute(query, object_data)
    connection.commit()
    
def delete_object(object_id, table):
    query_to_delete = f"""delete  from {table} where id = %s"""  
    cur.execute(query_to_delete, vars=(object_id, ))
    
def store_deleted_object(object_id, object_type, account_id, table):
    today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    query =  f"INSERT INTO {table} values (%s, %s, %s, %s)"
    cur.execute(query, vars=(object_id, account_id, object_type,today,))
    #connection.commit()

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    object_data['recording_date'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    return object_data

In [4]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

FacebookAdsApi.init(access_token=access_token)

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))

object_type = 'ad'
table = 'property_' + object_type + 's'

cur.execute(f""" CREATE TABLE if not exists {table}(
    id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     campaign_id VARCHAR(50), adset_id VARCHAR(50), 
    name VARCHAR(200), source_ad_id VARCHAR(50), source_ad JSONB, creative JSONB, adset JSONB, 
    campaign JSONB, bid_amount BIGINT, bid_info JSONB, bid_type VARCHAR(50),  
    created_time timestamp, updated_time timestamp, 
    effective_status VARCHAR(20), status VARCHAR(20), configured_status VARCHAR(20), 
    last_updated_by_app_id VARCHAR(50), tracking_specs JSONB, demolink_hash VARCHAR(50), 
    display_sequence INT, 
    preview_shareable_link VARCHAR(200), engagement_audience BOOLEAN, tracking_and_conversion_with_defaults JSONB, 
    conversion_specs JSONB, targeting JSONB, recommendations JSONB, ad_review_feedback JSONB, issues_info JSONB,
    failed_delivery_checks JSONB, adlabels JSONB,
    recording_date timestamp without time zone)""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists deleted_objects(
    object_id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     
    object_type VARCHAR(20),    recording_date timestamp without time zone)""")
connection.commit()

fields = list(Ad.Field.__dict__.keys())
exclude_list = ['__module__', '__doc__',  'adset_spec', 'audience_id', 'date_format', 
                'draft_adgroup_id', 'execution_options','is_autobid', 'include_demolink_hashes', 'filename']
fields= list(set(fields)-set(exclude_list))

query = """SELECT DISTINCT t.*
FROM (SELECT DISTINCT actions.object_id as ad_id, actions.account_id, 
property_ads.recording_date is NULL as has_not_data
FROM actions LEFT JOIN property_ads on actions.object_id = property_ads.id
WHERE object_type = 'ADGROUP' and  
(property_ads.recording_date is NULL or actions.event_time > property_ads.recording_date)
UNION
SELECT DISTINCT insights.ad_id, insights.account_id, False as has_not_data
from insights
left join property_ads on insights.ad_id = property_ads.id
where property_ads.id is null) t
LEFT JOIN deleted_objects on  t.ad_id = deleted_objects.object_id
WHERE deleted_objects.object_id IS NULL"""

ad_list = pd.read_sql(query, connection)
data = pd.read_sql(query, connection)
data = data[data['account_id'].isin(account_ids)]
ads = dict(zip(list(data['ad_id']), list(data['has_not_data'])))
accounts =  dict(zip(list(data['ad_id']), list(data['account_id'])))

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

for ad_id in tqdm(ads.keys()):
    try:
        ad_data = Ad(ad_id).api_get(fields = fields).export_all_data()
        ad_data = modify_object(ad_data)
        cur, connection = reconnection_to_database(host, port, database, user, password, connection)
        if ads[ad_id]:
            store_object(ad_data, table)
        else:
            delete_object(ad_id, table)
            store_object(ad_data, table)      
        
    except FacebookRequestError as error:
        code = error._body['error']['code']
        subcode = error._body['error'].get('error_subcode', None)
        if code == 100 and (subcode == 33 or subcode == 1487221):
            cur, connection = reconnection_to_database(host, port, database, user, password, connection)
            store_deleted_object(ad_id, object_type, accounts[ad_id], 'deleted_objects')
        print('error with ' + ad_id)

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-13 19:35:29


  0%|          | 0/4149 [00:00<?, ?it/s]

start uploading 	2020-07-13 19:35:36


  0%|          | 1/4149 [00:00<47:47,  1.45it/s]

error with 23842709547320537


...

error with 23845363061920006


100%|█████████▉| 4148/4149 [48:03<00:00,  1.65it/s]

error with 23845363067090006


100%|██████████| 4149/4149 [48:03<00:00,  1.44it/s]

error with 23845363067100006
end processing 	2020-07-13 20:23:39
